In [1]:
import pandas as pd
import numpy as np
import mlp_tensorflow as mlp_tf
import matplotlib.pyplot as plt

plt.style.use('seaborn')

In [2]:
data = pd.read_csv('data/Iris.csv')
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
mapping = {'Species': {'Iris-setosa': 1, 'Iris-versicolor': 2, 'Iris-virginica': 3}}
data.replace(mapping, inplace=True)

In [12]:
features = data.iloc[:,1:5].copy()
labels = data['Species']

0    1
1    1
2    1
3    1
4    1
Name: Species, dtype: int64

In [13]:
train_x = features[:120]
test_x = features[120:]

train_y = labels[:120]
test_y = labels[120:]

print('-'*30)
print('x :',train_x.shape, test_x.shape)
print('y :',train_y.shape, test_y.shape)
print('-'*30)

------------------------------
x : (120, 4) (30, 4)
y : (120,) (30,)
------------------------------


In [14]:
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')

train_y = train_y.astype('float32')
test_y = test_y.astype('float32')

In [6]:
import tensorflow as tf

In [7]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons

In [8]:
n_input = 4  # iris dataset (img shape: 28*28) 
n_classes = 3 # 

In [15]:
# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

In [9]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


Instructions for updating:
Colocations handled automatically by placer.


In [10]:
# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [16]:

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits,
    labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(loss_op)

# Initializing the variables
init = tf.global_variables_initializer()

In [19]:
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(120/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            print(i)
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")


0


NameError: name 'mnist' is not defined

In [ ]:

# Test model
pred = tf.nn.softmax(logits)  # Apply softmax to logits
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))